In [1]:
import pandas as pd

graph_df = pd.read_csv('analytics.csv')



raining_list = []

not_raining_list = []

graph_df['date_time']=pd.to_datetime(graph_df['last_update']/1000, unit='s')

graph_df['date_time'].head()

graph_df['hours']= graph_df['date_time'].dt.hour


In [2]:
graph_df.head()

,Unnamed: 0,available_bike_stands,available_bikes,bike_stands,last_update,number,status,latest_weather,clouds,conditions,date,description,sunrise,sunset,temperature,wind,date_time,hours
0,0,4,27,31,1521126715000,1,OPEN,1521124200,40,Clouds,1521124200,scattered clouds,1521095903,1521138611,10.52,8.7,2018-03-15 15:11:55,15
1,1,1,30,31,1521127532000,1,OPEN,1521124200,40,Clouds,1521124200,scattered clouds,1521095903,1521138611,10.52,8.7,2018-03-15 15:25:32,15
2,2,20,0,20,1521126523000,2,OPEN,1521124200,40,Clouds,1521124200,scattered clouds,1521095903,1521138611,10.52,8.7,2018-03-15 15:08:43,15
3,3,20,0,20,1521127131000,2,OPEN,1521124200,40,Clouds,1521124200,scattered clouds,1521095903,1521138611,10.52,8.7,2018-03-15 15:18:51,15
4,4,18,2,20,1521126673000,3,OPEN,1521124200,40,Clouds,1521124200,scattered clouds,1521095903,1521138611,10.52,8.7,2018-03-15 15:11:13,15


In [3]:
sorted(pd.unique(graph_df.description).ravel())



['broken clouds',
 'clear sky',
 'few clouds',
 'fog',
 'light intensity drizzle',
 'light intensity drizzle rain',
 'light intensity shower rain',
 'light rain',
 'light shower snow',
 'light snow',
 'mist',
 'moderate rain',
 'scattered clouds',
 'shower rain',
 'shower sleet',
 'shower snow']

In [4]:
df_rain = graph_df[(graph_df['description']=='moderate rain')  ]
# df_rain
df_not_rain = graph_df[(graph_df['description']!='moderate rain')  ]
# df_rain
print(df_rain.shape[0]+df_not_rain.shape[0],graph_df.shape[0])


155900 155900


In [5]:
def make_availability_list(df):    
    numbers = df['number'].unique()   #create list of unique station numbers
    bikes_by_station_by_hour = {}    #will store avg available_bikes for each station by hour of day
    for i in numbers:
        bikes_by_station_by_hour[i]=[]

    bikes_by_station_by_hour[2]
    hours = df['hours'].unique()   #create list of 24 hours
    hours.sort()      #sort hours
    for i in hours:    #iterate over hours
        hour = df[df['hours']==i]    #for each hour create data frame with only that hour to calculate avg for that hour
        for j in numbers:    #iterate over unique station numbers
            avg_available_bikes = hour.groupby(['number'])['available_bikes'].mean()[j]    #get mean available_bikes for station j
            bikes_by_station_by_hour[j].append(avg_available_bikes)
    #         bikes_by_station_by_hour[j].append(i)
     #print(i)
    return bikes_by_station_by_hour

rain_list= make_availability_list(df_rain)
no_rain_list= make_availability_list(df_not_rain )
# print(l1,l2)

KeyError: 10

In [ ]:
#  bikes_by_station_by_hour[1]


In [ ]:
station_rain=rain_list[1]
station_no_rain=no_rain_list[1]


In [ ]:
df = pd.DataFrame(data ={'hour':hours,'rain':station_rain,'not rain':station_no_rain})

In [ ]:
group = df.groupby('hour')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
avg = group.sum()
print(avg)
avg.plot(kind='bar')
